In [1]:
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')
  print('There are %d GPU(s) available' % torch.cuda.device_count())
  print(torch.cuda.get_device_name(0))
else:
  device = torch.device('cpu')
  print('No GPU available, CPU instaed')

device

There are 1 GPU(s) available
NVIDIA A100 80GB PCIe


device(type='cuda')

In [2]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
import pandas as pd
import numpy as np
from copy import deepcopy
import re
from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split

import torch
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, AdamW, BertConfig

import numpy as np
import datetime
import time
import random
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix, \
                            precision_score, recall_score, accuracy_score, roc_auc_score, average_precision_score

/home/daozerova/.conda/envs/fl/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/daozerova/.conda/envs/fl/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/daozerova/.conda/envs/fl/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
lenta_df = pd.read_excel('lenta_finam.xlsx')
rbk_df = pd.read_excel('rbk_finam.xlsx')

lenta_df_ton = pd.read_csv('lenta_finam_unique_sentiment.xlsx')
rbk_df_ton = pd.read_csv('rbk_finam_unique_sentiment.xlsx')

In [5]:
lenta_df = lenta_df.merge(lenta_df_ton, left_on='title', right_on='title').drop(['Unnamed: 0'], axis=1)
rbk_df = rbk_df.merge(rbk_df_ton, left_on='title', right_on='title').drop(['Unnamed: 0'], axis=1)

In [6]:
lenta_df = lenta_df[['docid', 'title', 'rightcol', 'text'] + lenta_df.columns[18:].to_list()]
lenta_df.rename(columns={'docid': 'id', 'rightcol':'body'}, inplace=True)
lenta_df.head()

,id,title,body,text,TICKER,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,date_y,sentiment
0,1181788,В МИД России назвали сроки сертификации «Север...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,SBER,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00,отрицательный
1,1181788,В МИД России назвали сроки сертификации «Север...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,ROSN,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00,отрицательный
2,1181788,В МИД России назвали сроки сертификации «Север...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,MOEX,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00,отрицательный
3,1181788,В МИД России назвали сроки сертификации «Север...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,YNDX,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00,отрицательный
4,1181788,В МИД России назвали сроки сертификации «Север...,В МИД России назвали сроки сертификации «Север...,Фото: Nikolai Ryutin / Nord Stream 2 Евгения Н...,PIKK,60,20220103,10:00,"1,326.2","1,330.5","1,321.7","1,323.1","22,990",2022-01-03 10:00:00,отрицательный


In [7]:
rbk_df = rbk_df[['id', 'category', 'title', 'body', 'text'] + rbk_df.columns[19:].to_list()]
rbk_df['text']  = 'Категория - ' + rbk_df['category'] + '. ' + rbk_df['text']
rbk_df.drop(['category'], axis=1, inplace=True)
rbk_df.head()

,id,title,body,text,TICKER,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,date_y,sentiment
0,61d4b1139a79478571789993,Токаев ввел чрезвычайное положение в Мангистау...,Чрезвычайное положение вступит в силу 5 января...,Категория - Политика. Президент Казахстана Кас...,SMLT,60,20220105,00:00,"1,305.5","1,306.2","1,304.5","1,305.9","8,080",2022-01-05,отрицательный
1,61d4b1139a79478571789993,Токаев ввел чрезвычайное положение в Мангистау...,Чрезвычайное положение вступит в силу 5 января...,Категория - Политика. Президент Казахстана Кас...,YNDX,60,20220105,00:00,"1,305.5","1,306.2","1,304.5","1,305.9","8,080",2022-01-05,отрицательный
2,61d4b1139a79478571789993,Токаев ввел чрезвычайное положение в Мангистау...,Чрезвычайное положение вступит в силу 5 января...,Категория - Политика. Президент Казахстана Кас...,MTSS,60,20220105,00:00,"1,305.5","1,306.2","1,304.5","1,305.9","8,080",2022-01-05,отрицательный
3,61d4b1139a79478571789993,Токаев ввел чрезвычайное положение в Мангистау...,Чрезвычайное положение вступит в силу 5 января...,Категория - Политика. Президент Казахстана Кас...,GAZP,60,20220105,00:00,"1,305.5","1,306.2","1,304.5","1,305.9","8,080",2022-01-05,отрицательный
4,61d4b1139a79478571789993,Токаев ввел чрезвычайное положение в Мангистау...,Чрезвычайное положение вступит в силу 5 января...,Категория - Политика. Президент Казахстана Кас...,SBER,60,20220105,00:00,"1,305.5","1,306.2","1,304.5","1,305.9","8,080",2022-01-05,отрицательный


In [8]:
news_df = pd.concat([lenta_df, rbk_df])
news_df.sort_values(by=['date_y'], inplace=True)

In [9]:
news_df['TICKER'].replace({'PIKK': 'ПИК - строительство и продажа недвижимости', 'GAZP': 'Газпром - продажа природного газа', \
                        'SMLT': 'Самолет - строительство и продажа недвижимости', 'TCSG': 'Тинькофф банк', 'ROSN': ' Роснефть -  нефтегазовая компания',\
                        'YNDX': 'Яндекс - сайты, изображения, музыка, товары', 'MTSS': 'МТС - телекоммуникационные услуги', \
                        'MOEX': ' Московская Биржа - торги акциями', 'LKOH': 'Лукойл -  добыча и переработка нефти и газа', 'SBER': 'Сбербанк'}, inplace=True)
news_df['source'] = news_df.id.astype(str).str.len().apply(lambda x: 'РБК' if x == 24 else 'Лента.ру')
news_df.drop(['DATE', 'PER', 'id'], axis=1, inplace=True)

In [10]:
news_df['Новость'] = 'Источник новости - ' + news_df['source'] + '. ' + 'Тональность - ' + news_df['sentiment'] + '. ' + 'Заголовок - ' + news_df['title'] + '. '\
                        + 'Статья. ' + news_df['text'] + '. ' + 'Компания и оказываемые услуги. ' + news_df['TICKER'] 
news_df['Новость'].iloc[0]

'Источник новости - РБК. Тональность - отрицательный. Заголовок - Хакеры взломали две израильские газеты в годовщину смерти Сулеймани. Статья. Категория - Общество. Сайт израильской газеты The Jerusalem Post подвергся хакерской атаке в ночь на 3 января, сообщило издание. Именно в этот день два года назад был убит иранский генерал Касем Сулеймани. Хакеры взломали также Twitter-аккаунт газеты Maariv. В нем и на главной странице сайта The Jerusalem Post появилась одна и та\xa0же картинка. На ней изображено взрывающееся здание, похожее на ядерный исследовательский центр в израильском городе Димона. На фоне над зданием\xa0— надпись на английском и иврите: «Мы рядом с вами, когда вы об этом не думаете». В левой верхней части изображения\xa0— сжатая в кулак ладонь с перстнем на безымянном пальце (он напоминает кольцо, которое носил Сулеймани), откуда вылетает баллистическая ракета. Опубликованная в Twitter Maariv запись с картинкой\xa0удалена, сайт The Jerusalem Post работает в нормальном реж

In [11]:
news_df['price_change'] = news_df['CLOSE'].astype(str).str.replace(',', '').astype(float) - news_df['OPEN'].astype(str).str.replace(',', '').astype(float)
news_df['changed'] = (news_df['price_change'] == 0).astype(int)

news_df['price_change_bins'] = pd.qcut((news_df['CLOSE'].astype(str).str.replace(',', '').astype(float) - news_df['OPEN'].astype(str).str.replace(',', '').astype(float)) \
                                       / news_df['OPEN'].astype(str).str.replace(',', '').astype(float) * 100, q=30)
bins = news_df['price_change_bins'].value_counts().index
#news_df['price_change'].value_counts().sort_index()

news_df['price_change_bins'].value_counts()

(1.126, 1.668]                   13611
(0.35, 0.419]                    13594
(0.517, 0.659]                   13574
(-0.39, -0.328]                  13569
(-0.0893, -0.0515]               13551
(0.18, 0.233]                    13544
(-0.702, -0.562]                 13530
(-0.562, -0.469]                 13525
(-1.874, -1.247]                 13524
(-0.0148, 0.0192]                13519
(0.053, 0.0939]                  13519
(0.136, 0.18]                    13501
(-21.240000000000002, -1.874]    13501
(-0.168, -0.129]                 13496
(0.659, 0.86]                    13495
(-0.328, -0.268]                 13493
(-1.247, -0.929]                 13489
(0.0192, 0.053]                  13467
(-0.268, -0.214]                 13459
(0.233, 0.284]                   13458
(-0.929, -0.702]                 13454
(-0.129, -0.0893]                13450
(0.0939, 0.136]                  13443
(0.284, 0.35]                    13441
(-0.214, -0.168]                 13437
(0.86, 1.126]            

In [12]:
from collections import Counter

def drop_duplicates(txt):
    try:
        txt = re.split('\. |: |\n', txt)
        counter_txt = Counter(txt)
        if len(txt) < 2: return ''
        i = len(txt)-2
        while counter_txt[txt[i]] > 1:
            i-=1
    except:
        return ''
    if i==len(txt)-2:
        return " ".join(txt)
    return " ".join(txt[:i+1])

def replace_numbers(txt):
    phone_pattern = r'\b(\+)?([78]\d{10})\b'
    date_pattern = r'\d{1,2}\.\d{1,2}\.\d{2,4}'
    time_pattern = r'(\d{1,2}\:)?\d{1,2}\:\d{1,2}'
    nums_pattern = r'(\d{1,2} )?\d+(\,\d{1,2})?'
    trash_pattern = r'УКАЖИ'
    #utf_pattern = r'\xa0'

    new_text = re.sub(phone_pattern, ' номер_телефона ', txt)
    new_text = re.sub(date_pattern, 'дата ', new_text)
    new_text = re.sub(time_pattern, 'время ', new_text)
    new_text = re.sub(nums_pattern, 'число ', new_text)
    new_text = re.sub(trash_pattern, '', new_text)

    new_text = re.sub(r'[^\w\s]','', new_text).lower()
    new_text = re.sub(r'[\\]',' ', new_text).lower()
    return new_text

def full_describe_request(txt, add_feature_txt, add_txt):
    return txt + ' [' + str(add_feature_txt) + ' ]:' + str(add_txt)

def parse(series, features, text_col = 'Новость'):
    txt = '' if not isinstance(series[text_col], str) else series[text_col]
    uniq_txt = drop_duplicates(txt)

    for feature in features:
        uniq_txt = full_describe_request(uniq_txt, feature, series[feature])
    txt = replace_numbers(uniq_txt)
    return txt.strip("\n")

In [13]:
news_df['Новость'] = news_df.progress_apply(
    parse,
    args = (
     ['Новость'],
    ),
    axis = 1,
)

request_txts = news_df['Новость']

news_df['Новость'].iloc[0]

  0%|          | 0/404511 [00:00<?, ?it/s]

'источник новости  рбк тональность  отрицательный заголовок  хакеры взломали две израильские газеты в годовщину смерти сулеймани статья категория  общество сайт израильской газеты the jerusalem post подвергся хакерской атаке в ночь на число  января сообщило издание именно в этот день два года назад был убит иранский генерал касем сулеймани хакеры взломали также twitterаккаунт газеты maariv в нем и на главной странице сайта the jerusalem post появилась одна и та\xa0же картинка на ней изображено взрывающееся здание похожее на ядерный исследовательский центр в израильском городе димона на фоне над зданием\xa0 надпись на английском и иврите мы рядом с вами когда вы об этом не думаете в левой верхней части изображения\xa0 сжатая в кулак ладонь с перстнем на безымянном пальце он напоминает кольцо которое носил сулеймани откуда вылетает баллистическая ракета опубликованная в twitter maariv запись с картинкой\xa0удалена сайт the jerusalem post работает в нормальном режиме как отмечает издание 

In [14]:
len(news_df['price_change_bins'].value_counts().index.codes)

30

In [15]:
news_df['price_change_bins'].values.codes

array([26, 26, 26, ..., 11, 11, 11], dtype=int8)

In [16]:
sentences= news_df['Новость'].values
labels =  news_df['price_change_bins'].values.codes #news_df['price_change_bins'].values

## Data preparing ##

In [17]:
import os

tokenizer = AutoTokenizer.from_pretrained(
    "ai-forever/sbert_large_nlu_ru",
    do_lower_case=True,
)

replace_tokens = {
        '[unused1]': 'номер_телефона',
        '[unused2]': 'дата',
        '[unused3]': 'время',
        '[unused4]': 'число',
}

vocab = tokenizer.get_vocab()

for old_token, new_token in replace_tokens.items():
    idx = vocab[old_token]
    del vocab[old_token]
    vocab[new_token] = idx

vocab = pd.Series(vocab).sort_values()
vocab = vocab.index.tolist()

with open('vocab.tmp', 'w', encoding = 'utf-8') as tmp_vocab_file:
    tmp_vocab_file.write('\n'.join(vocab))

tokenizer = AutoTokenizer.from_pretrained(
    "ai-forever/sbert_large_nlu_ru",
    vocab_file = 'vocab.tmp', do_lower_case=True, do_basic_tokenize=False
)

os.remove('vocab.tmp')

In [18]:
print('Original: ', sentences[0])
print('Tokenised: ', tokenizer.tokenize(sentences[0], add_special_tokens=True))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0], add_special_tokens=True)))

Original:  источник новости  рбк тональность  отрицательный заголовок  хакеры взломали две израильские газеты в годовщину смерти сулеймани статья категория  общество сайт израильской газеты the jerusalem post подвергся хакерской атаке в ночь на число  января сообщило издание именно в этот день два года назад был убит иранский генерал касем сулеймани хакеры взломали также twitterаккаунт газеты maariv в нем и на главной странице сайта the jerusalem post появилась одна и та же картинка на ней изображено взрывающееся здание похожее на ядерный исследовательский центр в израильском городе димона на фоне над зданием  надпись на английском и иврите мы рядом с вами когда вы об этом не думаете в левой верхней части изображения  сжатая в кулак ладонь с перстнем на безымянном пальце он напоминает кольцо которое носил сулеймани откуда вылетает баллистическая ракета опубликованная в twitter maariv запись с картинкой удалена сайт the jerusalem post работает в нормальном режиме как отмечает издание не

In [19]:
max_len = 0
input_ids_all = []

for sent in tqdm(sentences):
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
    input_ids_all.append(input_ids)

max_len

  0%|          | 0/404511 [00:00<?, ?it/s]

11313

In [20]:
input_ids = []
attention_masks = []

for sent in tqdm(sentences):
    encoded_dict = tokenizer.encode_plus(
        sent,
        add_special_tokens=True,
        max_length=256,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print(sentences[0])
print(input_ids[0])

  0%|          | 0/404511 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/daozerova/.conda/envs/fl/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


источник новости  рбк тональность  отрицательный заголовок  хакеры взломали две израильские газеты в годовщину смерти сулеймани статья категория  общество сайт израильской газеты the jerusalem post подвергся хакерской атаке в ночь на число  января сообщило издание именно в этот день два года назад был убит иранский генерал касем сулеймани хакеры взломали также twitterаккаунт газеты maariv в нем и на главной странице сайта the jerusalem post появилась одна и та же картинка на ней изображено взрывающееся здание похожее на ядерный исследовательский центр в израильском городе димона на фоне над зданием  надпись на английском и иврите мы рядом с вами когда вы об этом не думаете в левой верхней части изображения  сжатая в кулак ладонь с перстнем на безымянном пальце он напоминает кольцо которое носил сулеймани откуда вылетает баллистическая ракета опубликованная в twitter maariv запись с картинкой удалена сайт the jerusalem post работает в нормальном режиме как отмечает издание непонятно нах

In [21]:
len(input_ids), len(attention_masks), len(labels)

(404511, 404511, 404511)

In [3]:
# pd.DataFrame(input_ids, columns = [f'f_{i}' for i in range(input_ids.shape[1])]).to_parquet('data/input_ids.parquet', index=False, compression='gzip')
# pd.DataFrame(attention_masks, columns = [f'f_{i}' for i in range(attention_masks.shape[1])]).to_parquet('data/attention_masks.parquet', index=False, compression='gzip')
# pd.DataFrame(labels, columns = ['label']).to_parquet('data/labels.parquet', index=False, compression='gzip')

input_ids = torch.tensor(pd.read_parquet('data/input_ids.parquet').values)
attention_masks = torch.tensor(pd.read_parquet('data/attention_masks.parquet').values)
labels = torch.tensor(pd.get_dummies(pd.read_parquet('data/labels.parquet')['label']).values)

In [4]:
input_ids.shape, attention_masks.shape, labels.shape

(torch.Size([404511, 256]),
 torch.Size([404511, 256]),
 torch.Size([404511, 30]))

In [5]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.8 * len(dataset))
val_size = int(0.3 * (len(dataset) - train_size))
test_size = len(dataset) - train_size - val_size

train_data, val_data = random_split(dataset, [train_size, val_size + test_size])
val_data, test_data = random_split(val_data, [val_size, test_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} testing samples'.format(test_size))

323,608 training samples
24,270 validation samples
56,633 testing samples


In [6]:
batch_size = 32

train_dataloader = DataLoader(
    train_data,
    sampler = RandomSampler(train_data),
    batch_size=batch_size
)

validation_dataloader = DataLoader(
    val_data,
    sampler = SequentialSampler(val_data),
    batch_size=batch_size
)

## BERT prepare ##

In [7]:
import torch

t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

a, b = torch.cuda.mem_get_info()
print(a / 1024 / 1024 / 1024)
print(b / 1024 / 1024 / 1024)

78.842529296875
79.255615234375


In [8]:
#model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

model = BertForSequenceClassification.from_pretrained(
    "ai-forever/sbert_large_nlu_ru", #'bert-base-uncased',
    num_labels = 30,
    output_attentions = False,
    output_hidden_states = False,
)

model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024

In [9]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                  )

/home/daozerova/.conda/envs/fl/lib/python3.9/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
epochs = 2

total_steps = len(train_dataloader) * epochs
scheduler =  get_linear_schedule_with_warmup(optimizer,
                                             num_warmup_steps = 0,
                                             num_training_steps = total_steps)

## Training loop ##

**Training**:
*   Распаковать обучающие данные и лейблы
*   Загрузить данные на GPU для ускорения
*   Занулить градиенты с предыдущего шага
*   Forward pass (скормить данные в нейросеть и пробросить их вперед)
*   Backward pass (back propagation - посчитать градиенты по всем параметрам с помощью обратного распространения ошибки)
*   Обновить параметры с помощью optimizer.step()
*   Посчитать статистики, чтобы следить за обучением ⛅

**Evaluation**:

*   Распаковать валидационные данные и лейблы
*   Загрузить данные на GPU для ускорения
*   Forward pass (скормить данные в нейросеть и пробросить вперед)
*   Посчиатть loss и статистики для валиданционных данных, чтобы следить за обучением

In [11]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

a, b = torch.cuda.mem_get_info()
print(a / 1024 / 1024 / 1024)
print(b / 1024 / 1024 / 1024)

77.240966796875
79.255615234375


In [12]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
 
def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_t0 = time.time()

for epoch_i in range(0, epochs):
    print('')
    print('======= EPOCH {:} / {:} ======'.format(epoch_i + 1, epochs))
    print('Training...')
  
    t0 = time.time()
    total_train_loss = 0
    model.train()
  
    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            #Report progress
            print('   Batch {0:>5} of {1:>5}. Elapsed: {2:} Loss: {3:.4f}  Average_Train_Loss: {4:.4f}'.format(step, len(train_data), elapsed, res['loss'], total_train_loss/(batch_size * (step + 1))))
      
        #batch  contains 3 pytorch tensors: [0] input ids, [1] aattention masks, [2] labels
        # print(batch[0].shape)
        #print(batch[0])
        # print(batch[1].shape)
        # print(batch[2].shape)
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
    
        model.zero_grad()
        #Forward pass
        res = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,labels=b_labels.float())
        loss = res['loss']
        logits = res['logits'] #вероятности классов для батча
    
        total_train_loss += loss.item()
        loss.backward()
        #нормы градиентов обрезаем до 1.0, чтобы предотвратить проблему взрывающихся градиентов
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
        optimizer.step() #обновление весов
        scheduler.step()
    
        avg_train_loss = total_train_loss/len(train_dataloader)
        train_time = format_time(time.time() - t0)
    
    print(' Average training loss: {0:.2f}'.format(avg_train_loss))
    print(' Training epoch took: {:}'.format(train_time))
  
  
    #Validation
    print("Running Validation...")
  
    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
  
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
  
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            result = model(b_input_ids,
                          token_type_ids=None,
                          attention_mask=b_input_mask,
                          labels=b_labels,
                          return_dict=True)
        loss = result.loss
        logits = result.logits
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
  
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
  
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
  
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
  
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': train_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======= EPOCH 1 / 2 ======
Training...
   Batch    40 of 323608. Elapsed: 0:00:42 Loss: 0.2253  Average_Train_Loss: 0.0106
   Batch    80 of 323608. Elapsed: 0:01:25 Loss: 0.1618  Average_Train_Loss: 0.0082
   Batch   120 of 323608. Elapsed: 0:02:08 Loss: 0.1489  Average_Train_Loss: 0.0071
   Batch   160 of 323608. Elapsed: 0:02:51 Loss: 0.1460  Average_Train_Loss: 0.0065
   Batch   200 of 323608. Elapsed: 0:03:33 Loss: 0.1477  Average_Train_Loss: 0.0061
   Batch   240 of 323608. Elapsed: 0:04:16 Loss: 0.1477  Average_Train_Loss: 0.0059
   Batch   280 of 323608. Elapsed: 0:04:58 Loss: 0.1467  Average_Train_Loss: 0.0057
   Batch   320 of 323608. Elapsed: 0:05:41 Loss: 0.1485  Average_Train_Loss: 0.0055
   Batch   360 of 323608. Elapsed: 0:06:23 Loss: 0.1474  Average_Train_Loss: 0.0054
   Batch   400 of 323608. Elapsed: 0:07:06 Loss: 0.1467  Average_Train_Loss: 0.0054
   Batch   440 of 323608. Elapsed: 0:07:49 Loss: 0.1472  Average_Train_Loss: 0.0053
   Batch   480 of 323608. Elapsed: 0